<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/20250710_2%EB%B2%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -o "/content/drive/MyDrive/data.zip" -d "/content/data_1"

Archive:  /content/drive/MyDrive/data.zip
  inflating: /content/data_1/sample_submission.csv  
  inflating: /content/data_1/test.csv  
  inflating: /content/data_1/train.csv  


In [ ]:
# ==============================================================================
# ✅ 초기 환경 설정 및 라이브러리 설치 (가장 먼저 실행)
# ==============================================================================

# 1. pip 캐시를 모두 비웁니다.
print("Clearing pip cache...")
!pip cache purge

# 2. numpy와 scikit-learn 관련 패키지를 강제로 제거합니다.
print("Uninstalling numpy and scikit-learn related packages...")
!pip freeze | grep -E 'numpy|scikit-learn' | xargs -n 1 pip uninstall -y

# 3. Colab 기본 라이브러리 (TensorFlow, Numba 등)와 호환되는 특정 버전의 numpy와 scikit-learn을 설치합니다.
#    numpy를 1.26.x 대로 낮춰서 시도합니다.
print("Installing compatible versions of numpy and scikit-learn (attempting numpy 1.26.x)...")
!pip install numpy==1.26.4 scikit-learn==1.6.0 --no-cache-dir --quiet

# 4. 나머지 필요한 라이브러리들을 설치합니다.
print("Installing other necessary libraries...")
!pip install rdkit-pypi catboost xgboost lightgbm optuna --no-cache-dir --quiet

# 5. 파이썬 런타임을 강제로 재시작합니다. (가장 중요!)
print("Forcing runtime restart... Please wait a few seconds for reconnection.")
import os
os.kill(os.getpid(), 9)

# ==============================================================================

Clearing pip cache...
Files removed: 0
Uninstalling numpy and scikit-learn related packages...
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: scikit-learn 1.6.0
Uninstalling scikit-learn-1.6.0:
  Successfully uninstalled scikit-learn-1.6.0
Installing compatible versions of numpy and scikit-learn (attempting numpy 1.26.x)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 267.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 46.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=

In [ ]:
# import pandas as pd
# import os

# # ✅ Colab 경로
# extract_path = "/content/data_1"
# def get_path(filename):
#     return os.path.join(extract_path, filename)

# # ✅ 데이터 불러오기
# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))

# # ✅ 컬럼 확인
# print("✅ [train.csv] 컬럼 목록:")
# print(train.columns.tolist())

# print("\n✅ [test.csv] 컬럼 목록:")
# print(test.columns.tolist())

# # ✅ 샘플 데이터 미리 보기
# print("\n✅ [train.csv] 데이터 샘플:")
# print(train.head())

# print("\n✅ [test.csv] 데이터 샘플:")
# print(test.head())


In [ ]:
# # ✅ import
# import pandas as pd
# import numpy as np
# import os
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors, AllChem
# from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostRegressor, Pool
# from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import KFold

# # ✅ 경로 설정 (Colab 환경)
# extract_path = "/content/data_1"
# def get_path(filename):
#     return os.path.join(extract_path, filename)

# # ✅ 데이터 로딩
# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))
# submission = pd.read_csv(get_path("sample_submission.csv"))

# # ✅ RDKit Feature 추출
# def extract_rdkit_features(df):
#     mols = [Chem.MolFromSmiles(smi) for smi in df['Canonical_Smiles']]
#     features = {
#         'MolWt': [Descriptors.MolWt(mol) if mol else np.nan for mol in mols],
#         'LogP': [Crippen.MolLogP(mol) if mol else np.nan for mol in mols],
#         'NumHDonors': [Lipinski.NumHDonors(mol) if mol else np.nan for mol in mols],
#         'NumHAcceptors': [Lipinski.NumHAcceptors(mol) if mol else np.nan for mol in mols],
#         'TPSA': [rdMolDescriptors.CalcTPSA(mol) if mol else np.nan for mol in mols],
#         'NumRotatableBonds': [Descriptors.NumRotatableBonds(mol) if mol else np.nan for mol in mols],
#         'RingCount': [mol.GetRingInfo().NumRings() if mol else np.nan for mol in mols],
#         'HeavyAtomCount': [mol.GetNumHeavyAtoms() if mol else np.nan for mol in mols],
#         'FractionCSP3': [rdMolDescriptors.CalcFractionCSP3(mol) if mol else np.nan for mol in mols],
#         'NumAliphaticRings': [rdMolDescriptors.CalcNumAliphaticRings(mol) if mol else np.nan for mol in mols],
#         'NumAromaticRings': [rdMolDescriptors.CalcNumAromaticRings(mol) if mol else np.nan for mol in mols]
#     }
#     return pd.DataFrame(features)

# # ✅ Morgan Fingerprint 추출 (2048-bit)
# def get_morgan_fingerprint(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol:
#         fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
#         return np.array(fp)
#     else:
#         return np.zeros(nBits)

# def extract_morgan_df(df, nBits=2048):
#     fps = df['Canonical_Smiles'].apply(lambda x: get_morgan_fingerprint(x, nBits=nBits))
#     return pd.DataFrame(fps.tolist(), columns=[f'MFP_{i}' for i in range(nBits)])

# # ✅ Feature 생성
# X_train_rdkit = extract_rdkit_features(train)
# X_test_rdkit = extract_rdkit_features(test)
# X_train_morgan = extract_morgan_df(train)
# X_test_morgan = extract_morgan_df(test)

# # ✅ 결합
# X_train = pd.concat([X_train_rdkit, X_train_morgan], axis=1)
# X_test = pd.concat([X_test_rdkit, X_test_morgan], axis=1)
# y_train = train["Inhibition"]

# # ✅ 결측치 처리
# X_train.fillna(0, inplace=True)
# X_test.fillna(0, inplace=True)

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# # ✅ Sample Weight
# sample_weight = np.log1p(y_train)

# # ✅ Stacking 앙상블
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# oof_cat = np.zeros(len(X_train))
# oof_lgb = np.zeros(len(X_train))
# oof_xgb = np.zeros(len(X_train))
# test_cat = np.zeros(len(X_test))
# test_lgb = np.zeros(len(X_test))
# test_xgb = np.zeros(len(X_test))

# for train_idx, val_idx in kf.split(X_train_df):
#     X_tr, X_val = X_train_df.iloc[train_idx], X_train_df.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
#     w_tr = sample_weight.iloc[train_idx]

#     # CatBoost
#     cat = CatBoostRegressor(iterations=1500, learning_rate=0.03, depth=6,
#                             verbose=0, early_stopping_rounds=100, random_state=42)
#     cat.fit(Pool(X_tr, y_tr, weight=w_tr), eval_set=Pool(X_val, y_val))
#     oof_cat[val_idx] = cat.predict(X_val)
#     test_cat += cat.predict(X_test_df) / kf.n_splits

#     # LightGBM
#     lgb = LGBMRegressor(n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42)
#     lgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_lgb[val_idx] = lgb.predict(X_val)
#     test_lgb += lgb.predict(X_test_df) / kf.n_splits

#     # XGBoost
#     xgb = XGBRegressor(n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42)
#     xgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_xgb[val_idx] = xgb.predict(X_val)
#     test_xgb += xgb.predict(X_test_df) / kf.n_splits

# # ✅ 메타 모델
# stacked_train = np.vstack([oof_cat, oof_lgb, oof_xgb]).T
# stacked_test = np.vstack([test_cat, test_lgb, test_xgb]).T
# meta_model = Ridge(alpha=1.0)
# meta_model.fit(stacked_train, y_train)
# final_preds = meta_model.predict(stacked_test)

# # ✅ 제출 파일 저장
# submission['Inhibition'] = final_preds
# submission.to_csv('submission_stacking_rdkit_morgan.csv', index=False)
# print("✅ 'submission_stacking_rdkit_morgan.csv' 생성 완료!")


In [ ]:
# import pandas as pd
# import numpy as np
# import os
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors, AllChem
# from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostRegressor, Pool

# # ✅ 데이터 로딩
# extract_path = "/content/data_1"  # Colab 기준
# train = pd.read_csv(os.path.join(extract_path, "train.csv"))
# test = pd.read_csv(os.path.join(extract_path, "test.csv"))
# submission = pd.read_csv(os.path.join(extract_path, "sample_submission.csv"))

# # ✅ RDKit 기반 화학 특성 추출 (13개 내외)
# def extract_physchem_features(df):
#     mols = [Chem.MolFromSmiles(smi) for smi in df['Canonical_Smiles']]
#     features = {
#         'MolWt': [Descriptors.MolWt(mol) if mol else np.nan for mol in mols],
#         'LogP': [Crippen.MolLogP(mol) if mol else np.nan for mol in mols],
#         'NumHDonors': [Lipinski.NumHDonors(mol) if mol else np.nan for mol in mols],
#         'NumHAcceptors': [Lipinski.NumHAcceptors(mol) if mol else np.nan for mol in mols],
#         'TPSA': [rdMolDescriptors.CalcTPSA(mol) if mol else np.nan for mol in mols],
#         'NumRotatableBonds': [Descriptors.NumRotatableBonds(mol) if mol else np.nan for mol in mols],
#         'RingCount': [mol.GetRingInfo().NumRings() if mol else np.nan for mol in mols],
#         'HeavyAtomCount': [mol.GetNumHeavyAtoms() if mol else np.nan for mol in mols],
#         'FractionCSP3': [rdMolDescriptors.CalcFractionCSP3(mol) if mol else np.nan for mol in mols],
#         'NumAliphaticRings': [rdMolDescriptors.CalcNumAliphaticRings(mol) if mol else np.nan for mol in mols],
#         'NumAromaticRings': [rdMolDescriptors.CalcNumAromaticRings(mol) if mol else np.nan for mol in mols],
#     }
#     return pd.DataFrame(features)

# # ✅ Morgan Fingerprint (2048 bit)
# def get_morgan_fp(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol:
#         return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits))
#     else:
#         return np.zeros(nBits)

# def extract_morgan_df(df, nBits=2048):
#     fps = df['Canonical_Smiles'].apply(lambda x: get_morgan_fp(x, nBits=nBits))
#     return pd.DataFrame(fps.tolist(), columns=[f'MFP_{i}' for i in range(nBits)])

# # ✅ 피처 생성
# X_train_phys = extract_physchem_features(train)
# X_test_phys = extract_physchem_features(test)
# X_train_fp = extract_morgan_df(train)
# X_test_fp = extract_morgan_df(test)

# # ✅ 피처 결합
# X_train = pd.concat([X_train_phys, X_train_fp], axis=1)
# X_test = pd.concat([X_test_phys, X_test_fp], axis=1)
# y_train = train["Inhibition"]

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # ✅ Sample Weighting (log scale)
# sample_weight = np.log1p(y_train)

# # ✅ CatBoost 학습
# model = CatBoostRegressor(
#     iterations=1500,
#     learning_rate=0.03,
#     depth=6,
#     loss_function='RMSE',
#     early_stopping_rounds=100,
#     verbose=100,
#     random_state=42
# )

# model.fit(Pool(X_train_scaled, y_train, weight=sample_weight))
# preds = model.predict(X_test_scaled)

# # ✅ 제출 파일 저장
# submission["Inhibition"] = preds
# submission.to_csv("submission_catboost_morgan_physchem13.csv", index=False)
# print("✅ 'submission_catboost_morgan_physchem13.csv' 저장 완료")


In [ ]:
# # ✅ 라이브러리
# import os
# import numpy as np
# import pandas as pd
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors, AllChem
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from catboost import CatBoostRegressor, Pool
# from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor

# # ✅ 경로 설정 (Colab 기준)
# extract_path = "/content/data_1"
# def get_path(filename): return os.path.join(extract_path, filename)

# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))
# submission = pd.read_csv(get_path("sample_submission.csv"))

# # ✅ RDKit 기반 분자 특성 추출
# def extract_rdkit_features(df):
#     mols = [Chem.MolFromSmiles(smi) for smi in df['Canonical_Smiles']]
#     features = {
#         'MolWt': [Descriptors.MolWt(mol) if mol else np.nan for mol in mols],
#         'LogP': [Crippen.MolLogP(mol) if mol else np.nan for mol in mols],
#         'NumHDonors': [Lipinski.NumHDonors(mol) if mol else np.nan for mol in mols],
#         'NumHAcceptors': [Lipinski.NumHAcceptors(mol) if mol else np.nan for mol in mols],
#         'TPSA': [rdMolDescriptors.CalcTPSA(mol) if mol else np.nan for mol in mols],
#         'NumRotatableBonds': [Descriptors.NumRotatableBonds(mol) if mol else np.nan for mol in mols],
#         'RingCount': [mol.GetRingInfo().NumRings() if mol else np.nan for mol in mols],
#         'HeavyAtomCount': [mol.GetNumHeavyAtoms() if mol else np.nan for mol in mols],
#         'FractionCSP3': [rdMolDescriptors.CalcFractionCSP3(mol) if mol else np.nan for mol in mols],
#         'NumAliphaticRings': [rdMolDescriptors.CalcNumAliphaticRings(mol) if mol else np.nan for mol in mols],
#         'NumAromaticRings': [rdMolDescriptors.CalcNumAromaticRings(mol) if mol else np.nan for mol in mols]
#     }
#     return pd.DataFrame(features)

# # ✅ Morgan Fingerprint
# def get_morgan_fingerprint(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol:
#         fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
#         return np.array(fp)
#     else:
#         return np.zeros(nBits)

# def extract_morgan_df(df, nBits=2048):
#     fps = df['Canonical_Smiles'].apply(lambda x: get_morgan_fingerprint(x, nBits=nBits))
#     return pd.DataFrame(fps.tolist(), columns=[f'MFP_{i}' for i in range(nBits)])

# # ✅ Feature 결합
# X_train_rdkit = extract_rdkit_features(train)
# X_test_rdkit = extract_rdkit_features(test)
# X_train_morgan = extract_morgan_df(train)
# X_test_morgan = extract_morgan_df(test)

# X_train = pd.concat([X_train_rdkit, X_train_morgan], axis=1)
# X_test = pd.concat([X_test_rdkit, X_test_morgan], axis=1)
# y_train = train["Inhibition"]

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# # ✅ Sample Weight
# sample_weight = np.log1p(y_train)

# # ✅ 앙상블 초기화
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# oof_cat = np.zeros(len(X_train))
# oof_lgb = np.zeros(len(X_train))
# oof_xgb = np.zeros(len(X_train))
# test_cat = np.zeros(len(X_test))
# test_lgb = np.zeros(len(X_test))
# test_xgb = np.zeros(len(X_test))

# # ✅ 교차검증 모델 학습
# for train_idx, val_idx in kf.split(X_train_df):
#     X_tr, X_val = X_train_df.iloc[train_idx], X_train_df.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
#     w_tr = sample_weight.iloc[train_idx]

#     # ✅ CatBoost (GPU)
#     cat = CatBoostRegressor(
#         iterations=1500, learning_rate=0.03, depth=6, verbose=0,
#         early_stopping_rounds=100, random_state=42, task_type='GPU'
#     )
#     cat.fit(Pool(X_tr, y_tr, weight=w_tr), eval_set=Pool(X_val, y_val))
#     oof_cat[val_idx] = cat.predict(X_val)
#     test_cat += cat.predict(X_test_df) / kf.n_splits

#     # ✅ LightGBM (GPU)
#     lgb = LGBMRegressor(
#         n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42,
#         device='gpu'
#     )
#     lgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_lgb[val_idx] = lgb.predict(X_val)
#     test_lgb += lgb.predict(X_test_df) / kf.n_splits

#     # ✅ XGBoost (GPU)
#     xgb = XGBRegressor(
#         n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42,
#         tree_method='gpu_hist', predictor='gpu_predictor'
#     )
#     xgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_xgb[val_idx] = xgb.predict(X_val)
#     test_xgb += xgb.predict(X_test_df) / kf.n_splits

# # ✅ 메타 모델 (Ridge)
# stacked_train = np.vstack([oof_cat, oof_lgb, oof_xgb]).T
# stacked_test = np.vstack([test_cat, test_lgb, test_xgb]).T
# meta_model = Ridge(alpha=1.0)
# meta_model.fit(stacked_train, y_train)
# final_preds = meta_model.predict(stacked_test)

# # ✅ 성능 평가
# rmse = np.sqrt(mean_squared_error(y_train, meta_model.predict(stacked_train)))
# mae = mean_absolute_error(y_train, meta_model.predict(stacked_train))
# print(f"✅ Meta Model RMSE: {rmse:.4f}")
# print(f"✅ Meta Model MAE : {mae:.4f}")

# # ✅ 제출
# submission['Inhibition'] = final_preds
# submission.to_csv("submission_stacking_rdkit_morgan.csv", index=False)
# print("✅ 'submission_stacking_rdkit_morgan.csv' 저장 완료!")


In [4]:
# ✅ 전체 실행 코드
import os, gc, warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import optuna

# ✅ 경로 설정
extract_path = "/content/data_1"
def get_path(filename): return os.path.join(extract_path, filename)
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# ✅ RDKit Feature 생성
def calc_physchem(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return [np.nan] * 10
    return [
        Descriptors.MolWt(mol), Descriptors.MolLogP(mol), Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol), Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
        Descriptors.RingCount(mol), Descriptors.FractionCSP3(mol),
        Descriptors.HeavyAtomCount(mol), Descriptors.MolMR(mol)
    ]

physchem_cols = ['MolWt','LogP','HDonors','HAcceptors','TPSA','RotBonds','Rings','CSP3','HeavyAtoms','MR']
train_physchem = pd.DataFrame([calc_physchem(smi) for smi in train['Canonical_Smiles']], columns=physchem_cols)
test_physchem = pd.DataFrame([calc_physchem(smi) for smi in test['Canonical_Smiles']], columns=physchem_cols)

# ✅ Morgan Fingerprints
def morgan_fp(smiles, radius=2, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits) if mol else [0]*nBits

train_morgan = np.array([morgan_fp(smi) for smi in train['Canonical_Smiles']])
test_morgan = np.array([morgan_fp(smi) for smi in test['Canonical_Smiles']])

# ✅ Feature 결합 + 정규화
X_train = np.hstack([train_physchem.values, train_morgan])
X_test = np.hstack([test_physchem.values, test_morgan])
y_train = train["Inhibition"].values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ CatBoost Optuna 튜닝
def objective_catboost(trial):
    model = CatBoostRegressor(
        iterations=trial.suggest_int("iterations", 300, 700),
        depth=trial.suggest_int("depth", 4, 10),
        learning_rate=trial.suggest_float("lr", 0.01, 0.2),
        l2_leaf_reg=trial.suggest_float("l2", 1e-3, 10.0),
        loss_function="RMSE", task_type="GPU", verbose=0
    )
    scores = []
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    for tr_idx, val_idx in kf.split(X_train):
        model.fit(X_train[tr_idx], y_train[tr_idx])
        pred = model.predict(X_train[val_idx])
        scores.append(np.sqrt(mean_squared_error(y_train[val_idx], pred)))
    return np.mean(scores)

study = optuna.create_study(direction="minimize")
study.optimize(objective_catboost, n_trials=20)
best_cat_params = study.best_trial.params
best_cat_params_fixed = {
    'iterations': best_cat_params['iterations'],
    'depth': best_cat_params['depth'],
    'learning_rate': best_cat_params['lr'],
    'l2_leaf_reg': best_cat_params['l2']
}

# ✅ Base 모델 학습 + OOF
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_train = np.zeros((X_train.shape[0], 3))
oof_test = np.zeros((X_test.shape[0], 3))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"📦 Fold {fold+1}")
    cat = CatBoostRegressor(**best_cat_params_fixed, loss_function="RMSE", task_type="GPU", verbose=0)
    cat.fit(X_train[tr_idx], y_train[tr_idx])
    oof_train[val_idx, 0] = cat.predict(X_train[val_idx])
    oof_test[:, 0] += cat.predict(X_test) / kf.n_splits

    lgb = LGBMRegressor(device='gpu', learning_rate=0.05, n_estimators=500)
    lgb.fit(X_train[tr_idx], y_train[tr_idx])
    oof_train[val_idx, 1] = lgb.predict(X_train[val_idx])
    oof_test[:, 1] += lgb.predict(X_test) / kf.n_splits

    xgb = XGBRegressor(tree_method='gpu_hist', learning_rate=0.05, n_estimators=500)
    xgb.fit(X_train[tr_idx], y_train[tr_idx])
    oof_train[val_idx, 2] = xgb.predict(X_train[val_idx])
    oof_test[:, 2] += xgb.predict(X_test) / kf.n_splits

# ✅ Ridge Meta 모델 튜닝
def objective_meta(trial):
    alpha = trial.suggest_float("alpha", 0.01, 10.0)
    model = Ridge(alpha=alpha)
    scores = []
    kf_meta = KFold(n_splits=3, shuffle=True, random_state=42)
    for tr_meta, val_meta in kf_meta.split(oof_train):
        model.fit(oof_train[tr_meta], y_train[tr_meta])
        pred = model.predict(oof_train[val_meta])
        scores.append(np.sqrt(mean_squared_error(y_train[val_meta], pred)))
    return np.mean(scores)

meta_study = optuna.create_study(direction="minimize")
meta_study.optimize(objective_meta, n_trials=20)
best_alpha = meta_study.best_params["alpha"]

# ✅ 최종 예측 및 저장
meta_model = Ridge(alpha=best_alpha)
meta_model.fit(oof_train, y_train)
final_pred = meta_model.predict(oof_test)
submission["Inhibition"] = final_pred
submission.to_csv("submission_stacking_optuna_ridge.csv", index=False)
print("✅ 저장 완료: submission_stacking_optuna_ridge.csv")
print("✅ Meta RMSE:", mean_squared_error(y_train, meta_model.predict(oof_train), squared=False))
print("✅ Meta MAE :", mean_absolute_error(y_train, meta_model.predict(oof_train)))


[I 2025-07-10 06:23:48,216] A new study created in memory with name: no-name-bdc6b571-86f5-4c35-b9fc-ee28c998e152
[I 2025-07-10 06:24:19,228] Trial 0 finished with value: 25.042762396174243 and parameters: {'iterations': 681, 'depth': 8, 'lr': 0.09270997477968122, 'l2': 2.317004940100394}. Best is trial 0 with value: 25.042762396174243.
[I 2025-07-10 06:25:22,599] Trial 1 finished with value: 24.09810198132386 and parameters: {'iterations': 528, 'depth': 10, 'lr': 0.021628934095191194, 'l2': 3.47308957048379}. Best is trial 1 with value: 24.09810198132386.
[I 2025-07-10 06:25:28,752] Trial 2 finished with value: 23.94402793568152 and parameters: {'iterations': 336, 'depth': 7, 'lr': 0.10542357443836291, 'l2': 4.299417435894376}. Best is trial 2 with value: 23.94402793568152.
[I 2025-07-10 06:25:40,088] Trial 3 finished with value: 24.68024486252904 and parameters: {'iterations': 536, 'depth': 4, 'lr': 0.08941716103687852, 'l2': 3.3483199478494416}. Best is trial 2 with value: 23.944027

📦 Fold 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3509
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 770
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 25 dense feature groups (0.04 MB) transferred to GPU in 0.001606 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 33.391242
📦 Fold 2
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3488
[LightGBM] [Info] Number of data points in the train set: 1345, number of used features: 762
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin 

[I 2025-07-10 06:30:11,121] A new study created in memory with name: no-name-d0dcc222-88af-42df-ac3a-1e1acbe3848c
[I 2025-07-10 06:30:11,143] Trial 0 finished with value: 23.906869532514445 and parameters: {'alpha': 4.880541781535753}. Best is trial 0 with value: 23.906869532514445.
[I 2025-07-10 06:30:11,149] Trial 1 finished with value: 23.906865028066985 and parameters: {'alpha': 6.926444406041938}. Best is trial 1 with value: 23.906865028066985.
[I 2025-07-10 06:30:11,155] Trial 2 finished with value: 23.906878425879146 and parameters: {'alpha': 0.8475071295655999}. Best is trial 1 with value: 23.906865028066985.
[I 2025-07-10 06:30:11,160] Trial 3 finished with value: 23.906876238501354 and parameters: {'alpha': 1.8386833354763246}. Best is trial 1 with value: 23.906865028066985.
[I 2025-07-10 06:30:11,166] Trial 4 finished with value: 23.906864502539012 and parameters: {'alpha': 7.165277276111415}. Best is trial 4 with value: 23.906864502539012.
[I 2025-07-10 06:30:11,171] Trial 

✅ 저장 완료: submission_stacking_optuna_ridge.csv


TypeError: got an unexpected keyword argument 'squared'